## Bayesian Sets Recommender System - Maven

In [ ]:
# Algorithm Implementation: https://github.com/MaLL-UFSCar/bayessets
# Research Paper: 

In [1]:
!pip install git+https://github.com/MaLL-UFSCar/bayessets.git

  Cloning https://github.com/MaLL-UFSCar/bayessets.git to /tmp/pip-req-build-0cojoklw
  Running command git clone -q https://github.com/MaLL-UFSCar/bayessets.git /tmp/pip-req-build-0cojoklw
  Stored in directory: /tmp/pip-ephem-wheel-cache-lcai1tn_/wheels/17/99/fe/bc3c266fdef546aaf825421da93f0826c5eb36802c60c7247e
Successfully built BayesSets
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip show -f BayesSets

Name: BayesSets
Version: 0.2
Summary: Implementation of the Bayesian Sets algorithm and variants
Home-page: https://github.com/MaLL-UFSCar/bayessets
Author: Diorge Brognara
Author-email: diorge.bs@gmail.com
License: MIT
Location: /home/antrived/anaconda3/lib/python3.6/site-packages
Requires: scipy, numpy
Required-by: 
Files:
  BayesSets-0.2.dist-info/INSTALLER
  BayesSets-0.2.dist-info/METADATA
  BayesSets-0.2.dist-info/RECORD
  BayesSets-0.2.dist-info/WHEEL
  BayesSets-0.2.dist-info/top_level.txt
  bayessets/__init__.py
  bayessets/__pycache__/__init__.cpython-36.pyc
  bayessets/__pycache__/bayessets.cpython-36.pyc
  bayessets/bayessets.py


In [92]:
import numpy
import numpy as np
import pandas as pd
from scipy import sparse
import itertools
import re

In [4]:
# check working directory
import os
os.getcwd()

'/home/antrived/Redhat_Work/PycharmProjects/10-Bayesian-Sets-Recom-Maven'

### Data Loading & Pre-processing

In [5]:
# place 'package-to-id-dict-without-trans.json' and 'manifest-to-id-without-trans.pickle'
    # in wd from 'aagshah-pypi-insights' bucket

In [6]:
def cal_sparsity(array):
    num_total = total_elems(array)
    num_non_zero = zero_elems(array, num_total)
    sparsity = num_non_zero/num_total
    print("Sparsity of matrix is = {}".format(sparsity))
    return sparsity

def zero_elems(array, num_total):
    non_zero = numpy.count_nonzero(array)
    return num_total-non_zero

def total_elems(array):
    shape = array.shape
    return shape[0]*shape[1]

In [7]:
import json
import pickle

In [8]:
with open('package_id_dict.json', 'r') as f:
    pack_to_id = json.load(f)

In [9]:
with open('manifest_id_dict.json', 'rb') as f:
    man_to_id = json.load(f)

In [10]:
import copy
pack_to_id = copy.deepcopy(pack_to_id['package_list'])
pack_to_id

{'com.facebook.presto:presto-spi': 0,
 'com.facebook.presto:presto-resource-group-managers': 1,
 'com.facebook.presto:presto-array': 2,
 'com.facebook.presto:presto-plugin-toolkit': 3,
 'com.facebook.presto:presto-record-decoder': 4,
 'com.facebook.presto:presto-orc': 5,
 'com.facebook.presto:presto-rcfile': 6,
 'com.facebook.presto:presto-hive': 7,
 'com.facebook.presto:presto-hive-cdh4': 8,
 'com.facebook.presto:presto-example-http': 9,
 'com.facebook.presto:presto-local-file': 10,
 'com.teradata:re2j-td': 11,
 'com.facebook.presto:presto-tpch': 12,
 'com.facebook.presto:presto-blackhole': 13,
 'com.facebook.presto:presto-memory': 14,
 'com.facebook.presto:presto-base-jdbc': 15,
 'com.facebook.presto:presto-mysql': 16,
 'com.facebook.presto:presto-raptor': 17,
 'com.facebook.presto:presto-cli': 18,
 'com.facebook.presto:presto-bytecode': 19,
 'com.facebook.presto:presto-client': 20,
 'com.facebook.presto:presto-parser': 21,
 'com.facebook.presto:presto-main': 22,
 'com.facebook.prest

In [11]:
man_to_id = man_to_id['manifest_list']
man_to_id

{'0': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99],
 '1': [67,
  75,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120],
 '2': [128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  67,
  101,
  102,
  106,
  110,
  111,
  114,
  116,
  117,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127],
 '3': [135,
  140

In [12]:
users = len(man_to_id)
print(users) # 23115
items = len(pack_to_id)
print(items) # 19697

23115
19697


In [13]:
# initialize user-item matrix
rating_matrix = numpy.zeros((users, items))

In [14]:
# populate user-item matrix
for user, item_list in man_to_id.items():
    for item in item_list:
        rating_matrix[int(user)][item] = 1
# view matrix
print(rating_matrix.shape[0])
print(rating_matrix.shape[1])
rating_matrix[:3] # gets first 3 rows

23115
19697


array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
cal_sparsity(rating_matrix)
print(sum(sum(rating_matrix))) # no. of ratings = 866964.0

Sparsity of matrix is = 0.9980958240246944
866964.0


In [16]:
sparse_rating_matrix = sparse.csr_matrix(rating_matrix)
print(sparse_rating_matrix.shape[0]) # 
print(sparse_rating_matrix.shape[1]) # 

23115
19697


In [17]:
sparse_rating_matrix = sparse_rating_matrix.transpose()
print(sparse_rating_matrix.shape[0]) 
print(sparse_rating_matrix.shape[1])

19697
23115


In [18]:
sparse_rating_matrix[:10]

<10x23115 sparse matrix of type '<class 'numpy.float64'>'
	with 2360 stored elements in Compressed Sparse Column format>

### Training

In [19]:
import bayessets
model = bayessets.BernoulliBayesianSet(sparse_rating_matrix)

In [20]:
with open('Bayesian_Sets.pkl', 'wb') as f:
    pickle.dump(model, f)

In [21]:
with open('Bayesian_Sets.pkl', 'rb') as f:
    model = pickle.load(f)

### Scoring

In [22]:
import random
from math import ceil

def get_packages_from_id(package_ids):
    package_list = list()
    for i in package_ids:
        package = id_to_pack.get(str(i))
        package_list.append(package)
    return package_list

def map_input_to_package_ids(input_stack):
    package_id_list = list()
    for package in input_stack:
        package_id = pack_to_id.get(package)
        if package_id is not None:
            package_id_list.append(package_id)
    return package_id_list

# calculate probability of recommendatios
def calculate_recomm_prob(scores):
    normalized_scores=(scores-scores.min())/(scores.max()-scores.min())
    Prob = []
    for index in recommendations:
        Prob.append(int(100*normalized_scores[index]))
    return Prob

In [23]:
# with open('id-to-package-dict-without-trans.json', 'r') as f:
#     id_to_pack = json.load(f)

In [24]:
# create 'id_to_pack' dict
id_to_pack = {}
for key, val in pack_to_id.items():
    id_to_pack[str(val)] = key

In [39]:
input_stack = ['org.hibernate:hibernate-core']

In [ ]:
input_stack = [
 'net.sf.opencsv:opencsv',
 'org.apache.commons:commons-math3',
 'commons-codec:commons-codec',
 'io.netty:netty',
 'io.airlift.discovery:discovery-server',
 'com.amazonaws:aws-java-sdk-core',
 'com.amazonaws:aws-java-sdk-s3',
 'io.airlift:testing-mysql-server',
 'io.airlift:testing-postgresql-server',
 'org.apache.kafka:kafka_2.10',
 'org.xerial.snappy:snappy-java',
 'org.apache.zookeeper:zookeeper'
]

In [433]:
str1 = str(
"""

<?xml version="1.0" encoding="UTF-8"?>
<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/maven-v4_0_0.xsd">
	<modelVersion>4.0.0</modelVersion>
	<groupId>org.quicktheories</groupId>
	<artifactId>quicktheories-parent</artifactId>
	<version>0.27-SNAPSHOT</version>
	<name>quicktheories-parent</name>

	<organization>
		<name>org.quicktheories</name>
	</organization>
	<packaging>pom</packaging>

	<description>Property testing for java - parent project</description>
	<url>https://github.com/NCR-CoDE/QuickTheories</url>

	<licenses>
		<license>
			<name>Apache License, Version 2.0</name>
			<url>http://www.apache.org/licenses/LICENSE-2.0.txt</url>
			<distribution>repo</distribution>
		</license>
	</licenses>
	<scm>
		<connection>scm:git:https://github.com/NCR-CoDE/QuickTheories.git</connection>
		<developerConnection>scm:git:git@github.com:NCR-CoDE/QuickTheories.git</developerConnection>
		<url>https://github.com/NCR-CoDE/QuickTheories.git</url>
		<tag>HEAD</tag>
	</scm>
	<issueManagement>
		<url>https://github.com/NCR-CoDE/QuickTheories/issues</url>
		<system>GitHub</system>
	</issueManagement>

	<developers>
		<developer>
			<id>henry</id>
			<name>Henry Coles</name>
			<email>henry@pitest.org</email>
		</developer>
		<developer>
			<id>katy</id>
			<name>Katy Rae</name>
			<email>katy.rae@ncr.com</email>
		</developer>
	</developers>

	<properties>
		<project.build.sourceEncoding>UTF-8</project.build.sourceEncoding>
		<project.reporting.outputEncoding>UTF-8</project.reporting.outputEncoding>
		<bintray.repo>ncr-code/ncrcoe</bintray.repo>
		<bintray.package>QuickTheories</bintray.package>
	</properties>

	<modules>
		<module>core</module>
		<module>coverage</module>
	</modules>

	<build>
		<pluginManagement>
			<plugins>
				<plugin>
					<groupId>org.apache.maven.plugins</groupId>
					<artifactId>maven-compiler-plugin</artifactId>
					<version>3.8.0</version>
					<configuration>
						<source>1.8</source>
						<target>1.8</target>
					</configuration>
				</plugin>
				<plugin>
					<artifactId>maven-release-plugin</artifactId>
					<version>2.4.2</version>
					<dependencies>
						<dependency>
							<groupId>org.apache.maven.scm</groupId>
							<artifactId>maven-scm-provider-gitexe</artifactId>
							<version>1.9.4</version>
						</dependency>
					</dependencies>
					<configuration>
						<useReleaseProfile>false</useReleaseProfile>
						<releaseProfiles>release</releaseProfiles>
						<autoVersionSubmodules>true</autoVersionSubmodules>
					</configuration>
				</plugin>
				<plugin>
					<groupId>org.apache.maven.plugins</groupId>
					<artifactId>maven-surefire-plugin</artifactId>
					<version>2.22.1</version>
					<configuration>
						<excludes>
							<exclude>com/example/**.java</exclude>
							<exclude>**/*IT.java</exclude>
						</excludes>
						<argLine>-Xms512m -Xmx512m -XX:MaxMetaspaceSize=128m</argLine>
					</configuration>
				</plugin>
			</plugins>
		</pluginManagement>
	</build>

	<profiles>
		<profile>
			<id>release</id>
			<build>
				<plugins>
					<plugin>
						<artifactId>maven-source-plugin</artifactId>
						<version>2.4</version>
						<executions>
							<execution>
								<id>attach-sources</id>
								<goals>
									<goal>jar</goal>
								</goals>
							</execution>
						</executions>
					</plugin>
					<plugin>
						<artifactId>maven-javadoc-plugin</artifactId>
						<version>2.10.3</version>
						<executions>
							<execution>
								<id>attach-javadocs</id>
								<goals>
									<goal>jar</goal>
								</goals>
							</execution>
						</executions>
					</plugin>
				</plugins>
			</build>
		</profile>

	</profiles>

	<distributionManagement>
		<repository>
			<id>bintray-ncrcoe</id>
			<url>https://api.bintray.com/maven/${bintray.repo}/${bintray.package}/;publish=1</url>
		</repository>
	</distributionManagement>


	<!-- common dependencies used in all subprojects -->
	<dependencies>
		<dependency>
			<groupId>junit</groupId>
			<artifactId>junit</artifactId>
			<version>4.11</version>
			<scope>test</scope>
		</dependency>
		<dependency>
			<groupId>org.assertj</groupId>
			<artifactId>assertj-core</artifactId>
			<version>3.9.0</version>
			<scope>test</scope>
		</dependency>
		<dependency>
			<groupId>org.mockito</groupId>
			<artifactId>mockito-core</artifactId>
			<version>2.13.0</version>
			<scope>test</scope>
		</dependency>
		<dependency>
			<groupId>com.github.stefanbirkner</groupId>
			<artifactId>system-rules</artifactId>
			<version>1.13.0</version>
			<scope>test</scope>
		</dependency>
		<dependency>
			<groupId>nl.jqno.equalsverifier</groupId>
			<artifactId>equalsverifier</artifactId>
			<version>2.4.2</version>
			<scope>test</scope>
		</dependency>
	</dependencies>

</project>



""")

In [434]:
str1 = "".join(str1.split())
str1 = re.search('<dependencies>(.*)</dependencies>', str1)
str1 = str(str1.groups(1))

backup = str1

str1 = str1.split('</dependency>')
str1 = [str(x) for x in str1 if 'dependency' in x]

def get_pck(pckg_str):
    pckg_str = re.search('groupId>(.*)</artifactId', pckg_str)
    pckg_str = str(pckg_str.group(1))
    pckg_str = pckg_str.replace('</groupId><artifactId>',':')
    return pckg_str

input_stack = [get_pck(x) for x in str1]
input_stack = [x.split("<")[0] for x in input_stack]
input_stack

['org.apache.maven.scm:maven-scm-provider-gitexe',
 'org.apache.maven.plugins:maven-surefire-plugin',
 'org.assertj:assertj-core',
 'org.mockito:mockito-core',
 'com.github.stefanbirkner:system-rules',
 'nl.jqno.equalsverifier:equalsverifier']

In [435]:
len(input_stack)

6

In [436]:
input_stack = map_input_to_package_ids(input_stack)
print(input_stack)
print(len(input_stack))
# QC
print(get_packages_from_id(input_stack))
print(len(get_packages_from_id(input_stack)))

[1974, 11370, 89, 847, 12982, 1947]
6
['org.apache.maven.scm:maven-scm-provider-gitexe', 'org.apache.maven.plugins:maven-surefire-plugin', 'org.assertj:assertj-core', 'org.mockito:mockito-core', 'com.github.stefanbirkner:system-rules', 'nl.jqno.equalsverifier:equalsverifier']
6


In [437]:
# Scoring
scores = model.query(list(input_stack))
ranking = numpy.argsort(scores)[::-1]
top30 = ranking[:30]
recommendations = numpy.array(list(itertools.compress(top30,
    [i not in input_stack for i in top30])))
print(recommendations)
print(get_packages_from_id(recommendations))
print(calculate_recomm_prob(scores))

[1973 1964 1970 1963 1969 1654 1967 1975 1962 2236 1971 1966  541 1972
  542 1030  538  540 2234  543  544 2235   57  983 3527  850  985]
['org.apache.maven.scm:maven-scm-api', 'org.eclipse.jetty.alpn:alpn-api', 'com.barchart.udt:barchart-udt-bundle', 'org.eclipse.jetty.npn:npn-api', 'org.rxtx:rxtx', 'com.jcraft:jzlib', 'com.github.jponge:lzma-java', 'org.apache.ant:ant-launcher', 'org.jboss.marshalling:jboss-marshalling', 'com.ceilfors.maven.plugin:enforcer-rules', 'com.yammer.metrics:metrics-core', 'com.fasterxml:aalto-xml', 'org.jboss.narayana.jta:jdbc', 'org.tukaani:xz', 'org.jboss.narayana.jta:jms', 'net.jpountz.lz4:lz4', 'org.hamcrest:hamcrest-library', 'org.jboss:jboss-transaction-spi', 'org.mortbay.jetty.npn:npn-boot', 'org.jboss.narayana.jta:jta', 'org.jboss.narayana.jts:narayana-jts-integration', 'org.mortbay.jetty.alpn:alpn-boot', 'org.ow2.asm:asm-all', 'org.powermock:powermock-api-mockito', 'org.seleniumhq.selenium:selenium-remote-driver', 'org.skyscreamer:jsonassert', 'org

In [438]:
recom = get_packages_from_id(recommendations)
probs = calculate_recomm_prob(scores)
recom_prob = []
for i, prob in enumerate(probs):
    if prob >= 10:
#         recom_prob.append(str("{a}({b}%)".format( a=recom[i],b=probs[i] )))
        recom_prob.append(str("{a}".format( a=recom[i] )))

recom_prob

['org.apache.maven.scm:maven-scm-api',
 'org.eclipse.jetty.alpn:alpn-api',
 'com.barchart.udt:barchart-udt-bundle',
 'org.eclipse.jetty.npn:npn-api',
 'org.rxtx:rxtx',
 'com.jcraft:jzlib',
 'com.github.jponge:lzma-java',
 'org.apache.ant:ant-launcher',
 'org.jboss.marshalling:jboss-marshalling',
 'com.ceilfors.maven.plugin:enforcer-rules',
 'com.yammer.metrics:metrics-core',
 'com.fasterxml:aalto-xml',
 'org.jboss.narayana.jta:jdbc',
 'org.tukaani:xz',
 'org.jboss.narayana.jta:jms',
 'net.jpountz.lz4:lz4',
 'org.hamcrest:hamcrest-library',
 'org.jboss:jboss-transaction-spi',
 'org.mortbay.jetty.npn:npn-boot',
 'org.jboss.narayana.jta:jta',
 'org.jboss.narayana.jts:narayana-jts-integration',
 'org.mortbay.jetty.alpn:alpn-boot',
 'org.ow2.asm:asm-all',
 'org.powermock:powermock-api-mockito',
 'org.seleniumhq.selenium:selenium-remote-driver',
 'org.skyscreamer:jsonassert',
 'org.powermock:powermock-module-junit4']